In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amexfinal-r2/test_data.parquet
/kaggle/input/amexfinal-r2/add_event.parquet
/kaggle/input/amexfinal-r2/685404e30cfdb_submission_template.csv
/kaggle/input/amexfinal-r2/data_dictionary.csv
/kaggle/input/amexfinal-r2/offer_metadata.parquet
/kaggle/input/amexfinal-r2/add_trans.parquet
/kaggle/input/amexfinal-r2/train_data.parquet


In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os
os.listdir("/kaggle/input")


['amexfinal-r2']

In [4]:
BASE_PATH = "/kaggle/input/amexfinal-r2"
train_df = pd.read_parquet(f"{BASE_PATH}/train_data.parquet")
print(train_df.head())

                                               id1      id2        id3  \
0  1366776_189706075_16-23_2023-11-02 22:22:00.042  1366776  189706075   
1      1366776_89227_16-23_2023-11-01 23:51:24.999  1366776      89227   
2      1366776_35046_16-23_2023-11-01 00:30:59.797  1366776      35046   
3    1366776_6275451_16-23_2023-11-02 22:21:32.261  1366776    6275451   
4      1366776_78053_16-23_2023-11-02 22:21:34.799  1366776      78053   

                       id4         id5  y   f1    f2    f3    f4  ...  f357  \
0  2023-11-02 22:22:00.042  2023-11-02  0  1.0  None  None  None  ...  None   
1  2023-11-01 23:51:24.999  2023-11-01  0  1.0  None  None  None  ...  None   
2  2023-11-01 00:30:59.797  2023-11-01  0  1.0  None  None  None  ...  None   
3  2023-11-02 22:21:32.261  2023-11-02  0  1.0  None  None  None  ...  None   
4  2023-11-02 22:21:34.799  2023-11-02  0  1.0  None  None  None  ...  None   

      f358 f359  f360   f361 f362               f363    f364 f365  \
0  -9999.0 

In [5]:
import pandas as pd

# Load train and test datasets
train_df = pd.read_parquet(f"{BASE_PATH}/train_data.parquet")
test_df = pd.read_parquet(f"{BASE_PATH}/test_data.parquet")

# Load transaction and event logs
trans_df = pd.read_parquet(f"{BASE_PATH}/add_trans.parquet")
event_df = pd.read_parquet(f"{BASE_PATH}/add_event.parquet")

# Load offer metadata
offer_df = pd.read_parquet(f"{BASE_PATH}/offer_metadata.parquet")

# Load data dictionary (optional, for reference)
data_dict = pd.read_csv(f"{BASE_PATH}/data_dictionary.csv")


In [6]:
print("✅ Files Loaded")
print("Train:", train_df.shape)
print("Test:", test_df.shape)
print("Transactions:", trans_df.shape)
print("Events:", event_df.shape)
print("Offers:", offer_df.shape)


✅ Files Loaded
Train: (770164, 372)
Test: (369301, 371)
Transactions: (6339465, 9)
Events: (21457473, 5)
Offers: (4164, 12)


In [7]:
# Ensure 'id3' is of type string
for df in [train_df, test_df, offer_df]:
    df['id3'] = df['id3'].astype(str)

# Merge offer metadata
train_df = train_df.merge(offer_df, on='id3', how='left')
test_df = test_df.merge(offer_df, on='id3', how='left')

print("✅ Offer metadata merged.")


✅ Offer metadata merged.


In [8]:
# 🧾 Check what columns actually exist in your transaction file
print("Columns in trans_df:")
print(trans_df.columns.tolist())

print("\nSample rows:")
print(trans_df.head())


Columns in trans_df:
['id2', 'f367', 'f368', 'f369', 'f370', 'f371', 'f372', 'id8', 'f374']

Sample rows:
       id2   f367 f368 f369        f370      f371    f372       id8 f374
0  2896709  15.60  PBR    D  2023-10-16  19:16:52  202310  59639998  DSE
1  2855047   6.40   PR    D  2023-10-14  13:01:16  202310  59639998  DSE
2  2497175  13.99  PBR    D  2023-10-14  00:31:48  202310  59639998  DSE
3  2655364  15.14  PGC    D  2023-10-13  12:37:25  202310  59639998  DSE
4  2855047   2.12   PR    D  2023-10-09  16:51:21  202310  59639998  DSE


In [9]:
#Step 2.2 (Transaction Aggregation)
# Fix Step 2.2 — Transaction Aggregation for your dataset
trans_df['id2'] = trans_df['id2'].astype(str)
trans_df['f370'] = pd.to_datetime(trans_df['f370'])  # transaction datetime
trans_df['f372'] = trans_df['f372'].astype(str)       # month column (YYYYMM)

agg_trans = trans_df.groupby('id2').agg(
    total_spend=('f367', 'sum'),
    avg_spend=('f367', 'mean'),
    max_spend=('f367', 'max'),
    min_spend=('f367', 'min'),
    num_txns=('f367', 'count'),
    num_active_months=('f372', 'nunique'),
    most_common_industry=('f368', lambda x: x.mode().iloc[0] if not x.mode().empty else None)
).reset_index()


In [10]:
for df in [train_df, test_df]:
    df['id2'] = df['id2'].astype(str)

train_df = train_df.merge(agg_trans, on='id2', how='left')
test_df = test_df.merge(agg_trans, on='id2', how='left')

print("✅ Transaction features merged successfully.")

✅ Transaction features merged successfully.


In [11]:
event_df['id2'] = event_df['id2'].astype(str)
event_df['id4'] = pd.to_datetime(event_df['id4'])  # event datetime

# Extract hour and day
event_df['hour'] = event_df['id4'].dt.hour
event_df['day'] = event_df['id4'].dt.dayofweek  # 0=Monday, 6=Sunday

# Aggregate available features
agg_event = event_df.groupby('id2').agg(
    total_impressions=('id3', 'count'),
    most_common_hour=('hour', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    most_common_day=('day', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    first_event_date=('id4', 'min'),
    last_event_date=('id4', 'max')
).reset_index()

In [12]:
train_df = train_df.merge(agg_event, on='id2', how='left')
test_df = test_df.merge(agg_event, on='id2', how='left')

print("✅ Event features merged successfully (CTR skipped due to no click label).")


✅ Event features merged successfully (CTR skipped due to no click label).


In [13]:
#Step 3: Code for Preprocessing

from sklearn.preprocessing import LabelEncoder

# Label encode categorical features
for col in ['most_common_day', 'most_common_hour', 'most_common_industry']:
    if col in train_df.columns:
        le = LabelEncoder()
        combined = pd.concat([train_df[col], test_df[col]]).astype(str)
        le.fit(combined)
        train_df[col] = le.transform(train_df[col].astype(str))
        test_df[col] = le.transform(test_df[col].astype(str))

print("✅ Label encoding completed.")


✅ Label encoding completed.


In [14]:
print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

Train shape: (770164, 395)
Test shape: (369301, 394)


In [15]:
#Confirm Target Column
train_df['y'] = train_df['y'].astype(int)
print(train_df['y'].dtype)
print(train_df['y'].value_counts())


int64
y
0    733113
1     37051
Name: count, dtype: int64


In [16]:
#Step 4 — Model Training (LightGBM)

#4.1 — Select Final Features

features = [
    'f375', 'f376',
    'total_spend', 'avg_spend', 'max_spend', 'min_spend', 'num_txns',
    'num_active_months', 'most_common_industry',
    'total_impressions', 'most_common_hour', 'most_common_day'
]


In [17]:
#4.2 — Train-Test Split
from sklearn.model_selection import train_test_split

X = train_df[features].astype('float32')
y = train_df['y'].astype(int)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print("✅ Training size:", X_train.shape)
print("✅ Validation size:", X_val.shape)


✅ Training size: (616131, 12)
✅ Validation size: (154033, 12)


In [18]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

# Prepare LightGBM datasets
train_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_val, label=y_val)

# LightGBM Parameters
params = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'learning_rate': 0.05,
    'verbosity': -1,
    'seed': 42
}

# Train model
print("🚀 Training model...")
model = lgb.train(
    params,
    train_data,
    valid_sets=[val_data],
    num_boost_round=1000,
    callbacks=[lgb.early_stopping(50), lgb.log_evaluation(100)]
)

# Evaluate AUC on validation set
val_preds = model.predict(X_val)
val_auc = roc_auc_score(y_val, val_preds)
print("✅ Validation AUC Score:", round(val_auc, 6))


🚀 Training model...
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.616525
✅ Validation AUC Score: 0.616525


In [19]:
#Add More Relevant Features

#Hyperparameter Tuning

params = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'learning_rate': 0.01,
    'num_leaves': 64,
    'max_depth': 7,
    'min_child_samples': 100,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'scale_pos_weight': 20,  # class imbalance
    'seed': 42
}


In [20]:
#Use Stratified K-Fold CV
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
auc_scores = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    train_data = lgb.Dataset(X_train, label=y_train)
    val_data = lgb.Dataset(X_val, label=y_val)

    model = lgb.train(
        params, train_data,
        valid_sets=[val_data],
        num_boost_round=1000,
        callbacks=[lgb.early_stopping(50), lgb.log_evaluation(100)]
    )

    val_preds = model.predict(X_val)
    auc = roc_auc_score(y_val, val_preds)
    auc_scores.append(auc)
    print(f"Fold {fold+1} AUC: {auc:.5f}")

print("Average AUC:", np.mean(auc_scores))


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[38]	valid_0's auc: 0.614003
Fold 1 AUC: 0.61400
Training until validation scores don't improve for 50 rounds
[100]	valid_0's auc: 0.609917
Early stopping, best iteration is:
[64]	valid_0's auc: 0.609917
Fold 2 AUC: 0.60992
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[32]	valid_0's auc: 0.614315
Fold 3 AUC: 0.61432
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[30]	valid_0's auc: 0.606279
Fold 4 AUC: 0.60628
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[34]	valid_0's auc: 0.613119
Fold 5 AUC: 0.61312
Average AUC: 0.6115265030583193


In [21]:
#Phase 1: Feature Engineering Improvements

import numpy as np
import pandas as pd

# 1. Event span (days between first and last event)
for df in [train_df, test_df]:
    df['event_span_days'] = (
        (df['last_event_date'] - df['first_event_date']).dt.days
        if 'last_event_date' in df and 'first_event_date' in df else np.nan
    )

# 2. Days since last event (using a reference date: Nov 6, 2023)
reference_date = pd.to_datetime("2023-11-06")
for df in [train_df, test_df]:
    if 'last_event_date' in df:
        df['days_since_last_event'] = (reference_date - df['last_event_date']).dt.days

# 3. Transaction density: number of transactions per active month
for df in [train_df, test_df]:
    df['txn_per_month'] = df['num_txns'] / (df['num_active_months'] + 1e-5)

# 4. Flags for missing data
for df in [train_df, test_df]:
    df['has_trans_data'] = df['num_txns'].notna().astype(int)
    df['has_event_data'] = df['total_impressions'].notna().astype(int)

print("✅ Engineered new features.")


✅ Engineered new features.


In [22]:
features = [
    # Base
    'f375', 'f376',
    
    # Transactions
    'total_spend', 'avg_spend', 'max_spend', 'min_spend',
    'num_txns', 'num_active_months', 'txn_per_month',
    'most_common_industry',
    
    # Events
    'total_impressions', 'most_common_hour', 'most_common_day',
    'event_span_days', 'days_since_last_event',
    
    # Flags
    'has_trans_data', 'has_event_data'
]


In [23]:
#Phase 2: Hyperparameter Tuning (Manual Grid)

params = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'learning_rate': 0.02,
    'num_leaves': 64,
    'max_depth': 7,
    'min_child_samples': 100,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'reg_alpha': 0.1,
    'reg_lambda': 1.0,
    'scale_pos_weight': 20,
    'verbosity': -1,
    'seed': 42
}


In [24]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb

# 🧠 Feature list (update if you added more)
features = [
    'f375', 'f376',
    'total_spend', 'avg_spend', 'max_spend', 'min_spend',
    'num_txns', 'num_active_months', 'txn_per_month',
    'most_common_industry',
    'total_impressions', 'most_common_hour', 'most_common_day',
    'event_span_days', 'days_since_last_event',
    'has_trans_data', 'has_event_data'
]

# 🧪 Prepare input data
X = train_df[features].astype('float32')
y = train_df['y'].astype(int)

# ⚙️ Improved LightGBM parameters
params = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'learning_rate': 0.02,
    'num_leaves': 64,
    'max_depth': 7,
    'min_child_samples': 100,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'reg_alpha': 0.1,
    'reg_lambda': 1.0,
    'scale_pos_weight': 20,
    'verbosity': -1,
    'seed': 42
}

# 🔁 5-Fold Stratified CV
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
auc_scores = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    print(f"\n📂 Fold {fold + 1}")

    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    dtrain = lgb.Dataset(X_train, label=y_train)
    dval = lgb.Dataset(X_val, label=y_val)

    model = lgb.train(
        params,
        dtrain,
        valid_sets=[dval],
        num_boost_round=1000,
        callbacks=[lgb.early_stopping(50), lgb.log_evaluation(100)]
    )

    val_preds = model.predict(X_val)
    auc = roc_auc_score(y_val, val_preds)
    auc_scores.append(auc)
    print(f"✅ Fold {fold + 1} AUC: {auc:.5f}")

print("\n✅ Average AUC across folds:", round(np.mean(auc_scores), 6))



📂 Fold 1
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[16]	valid_0's auc: 0.614119
✅ Fold 1 AUC: 0.61412

📂 Fold 2
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[36]	valid_0's auc: 0.609813
✅ Fold 2 AUC: 0.60981

📂 Fold 3
Training until validation scores don't improve for 50 rounds
[100]	valid_0's auc: 0.614742
Early stopping, best iteration is:
[80]	valid_0's auc: 0.614745
✅ Fold 3 AUC: 0.61474

📂 Fold 4
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[18]	valid_0's auc: 0.606518
✅ Fold 4 AUC: 0.60652

📂 Fold 5
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[22]	valid_0's auc: 0.613235
✅ Fold 5 AUC: 0.61323

✅ Average AUC across folds: 0.611686


In [25]:
#Step 5.1 — Make Predictions on Test Data
# Ensure test features are ready
X_test = test_df[features].astype('float32')

# Use last model from Fold 5 (or retrain on full data for final)
test_preds = model.predict(X_test)
test_df['pred'] = test_preds


In [26]:
#Step 5.2 — Rank Offers Per Customer (id2)

# Rank id3 offers by proba for each id2
ranked = test_df[['id2', 'id3', 'pred']].copy()
ranked['rank'] = ranked.groupby('id2')['pred'].rank(method='first', ascending=False)

# Filter top 7 offers per customer
top7 = ranked[ranked['rank'] <= 7].copy()
top7 = top7.sort_values(by=['id2', 'rank'])  # Sort for submission formatting


In [27]:
#Step 5.3 — Format Submission File
# Create submission dataframe in required format
submission = top7[['id2', 'id3']].copy()
submission['id'] = submission['id2'].astype(str) + '_' + submission['id3'].astype(str)
submission = submission[['id']]  # Final format: only one column 'id'


In [28]:
# Step 5.4 — Save Submission


submission.to_csv("submission.csv", index=False)
print("📁 submission.csv saved successfully!")
submission.head(10)


📁 submission.csv saved successfully!


,id
153767,1000061_5420674
126858,1000061_803982
240303,1000061_97081
279821,1000061_97904824
52697,1000061_430736
167476,1000061_80107221
288480,1000061_31794
286069,1000131_67681
120958,1000131_74649
68256,1000131_415582


In [29]:
# ✅ STEP 1: Predict on test set
X_test = test_df[features].astype('float32')
test_df['pred'] = model.predict(X_test)

# ✅ STEP 2: Create final submission DataFrame in required format
submission = test_df[['id1','id2', 'id3', 'id5', 'pred']].copy()

test_df['rank'] = test_df.groupby('id2')['pred'].rank(method='first', ascending=False)
submission_df = test_df[test_df['rank'] <= 7].copy()

# ✅ STEP 3: Sort descending by probability (optional but useful for leaderboard check)

submission = submission_df[['id1','id2', 'id3', 'id5', 'pred']].copy() 
submission.info()
submission = submission.sort_values(by=['id2','pred'], ascending=False)
submission.info()
# Convert id5 to datetime (if not already)
submission['id5'] = pd.to_datetime(submission['id5'])

# Change format to MM/DD/YYYY (e.g., 11/5/2023)
submission['id5'] = submission['id5'].dt.strftime('%-m/%-d/%Y')

submission.head()

<class 'pandas.core.frame.DataFrame'>
Index: 79011 entries, 3 to 369300
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id1     79011 non-null  object 
 1   id2     79011 non-null  object 
 2   id3     79011 non-null  object 
 3   id5     79011 non-null  object 
 4   pred    79011 non-null  float64
dtypes: float64(1), object(4)
memory usage: 3.6+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 79011 entries, 258531 to 288480
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id1     79011 non-null  object 
 1   id2     79011 non-null  object 
 2   id3     79011 non-null  object 
 3   id5     79011 non-null  object 
 4   pred    79011 non-null  float64
dtypes: float64(1), object(4)
memory usage: 3.6+ MB


,id1,id2,id3,id5,pred
258531,1910565_55247_16-23_2023-11-04 15:57:55.266,1910565,55247,11/4/2023,0.213239
190109,1910523_65699008_16-23_2023-11-05 06:26:41.492,1910523,65699008,11/5/2023,0.261059
333709,1910523_735023_16-23_2023-11-05 06:26:42.053,1910523,735023,11/5/2023,0.261059
138226,1910523_65430_16-23_2023-11-04 08:24:17.803,1910523,65430,11/4/2023,0.213239
122183,1910523_57342_16-23_2023-11-04 08:24:17.804,1910523,57342,11/4/2023,0.185744


In [30]:
# test_df.info()
# submission.info()
submission_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 79011 entries, 3 to 369300
Columns: 401 entries, id1 to rank
dtypes: datetime64[ns](2), float64(14), int64(5), object(380)
memory usage: 242.3+ MB


In [31]:
# ✅ STEP 4: Save to CSV
submission.to_csv("submission.csv", index=False)
print("📁 submission.csv saved in required format.")
submission.head()

📁 submission.csv saved in required format.


,id1,id2,id3,id5,pred
258531,1910565_55247_16-23_2023-11-04 15:57:55.266,1910565,55247,11/4/2023,0.213239
190109,1910523_65699008_16-23_2023-11-05 06:26:41.492,1910523,65699008,11/5/2023,0.261059
333709,1910523_735023_16-23_2023-11-05 06:26:42.053,1910523,735023,11/5/2023,0.261059
138226,1910523_65430_16-23_2023-11-04 08:24:17.803,1910523,65430,11/4/2023,0.213239
122183,1910523_57342_16-23_2023-11-04 08:24:17.804,1910523,57342,11/4/2023,0.185744


In [32]:
# STEP 1: Predict on full test set
X_test = test_df[features].astype('float32')
test_df['pred'] = model.predict(X_test)

# STEP 2: Create submission DataFrame with required columns
submission = test_df[['id1','id2','id3','id5', 'pred']].copy()

# ✅ Optional: Sort by descending probability
submission = submission.sort_values(by='pred', ascending=False)

# STEP 3: Save to CSV
submission.to_csv("submission_full.csv", index=False)
print("📁 submission_full.csv saved with", submission.shape[0], "rows.")
submission.head() 


📁 submission_full.csv saved with 369301 rows.


,id1,id2,id3,id5,pred
67205,1858080_189706075_16-23_2023-11-04 18:43:29.012,1858080,189706075,2023-11-04,0.276043
235218,1167053_189706075_16-23_2023-11-05 18:48:55.644,1167053,189706075,2023-11-05,0.276043
365084,1151820_876665835_16-23_2023-11-05 14:20:37.554,1151820,876665835,2023-11-05,0.276043
113175,1824429_60213892_16-23_2023-11-04 10:35:38.205,1824429,60213892,2023-11-04,0.276043
25173,1025069_3048949_16-23_2023-11-05 11:29:32.377,1025069,3048949,2023-11-05,0.276043
